<a href="https://colab.research.google.com/github/daniel-mueller92/llm_mail_classifier/blob/main/llm_classifier_Mixtral-8x7b-instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install huggingface_hub

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

import re
import json
import os

# dont do this at home
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_excel('/content/drive/MyDrive/dataset_anonymized.xlsx')

In [5]:
def generate_prompt(df, idx, step=1):

    file_content = df["txt"][idx]

    task1 = """Bitte extrahiere folgende Informationen aus dem Dokument:
    1. Um welche Kategorie von Dokument handelt es sich? Die Auswahlmöglichkeiten sind "Deckungszusage", "Deckungsablehnung", "Nachfrage" oder "Kostensache". Bei einer
    Deckungszusage, erklärt die Versicherung, sie wird die zukünftigen Kosten des Falls übernehmen - bei einer Deckungszusage verneint sie das. Bei einer Nachfrage
    möchte die Versicherung mehr Informationen von der Kanzlei. Eine Kostensache liegt nur vor, wenn die Versicherung erklärt, sie hat bereits Kosten bezahlt.
    2. Wie lautet das Aktenzeichen der Kanzlei?
    3. Wie lautet der Name des Mandanten?
    4. Wie lautet die Schadensnummer des Versicherungsfalls?
    Wenn du einen Wert nicht findest, schreibe "null".

    Bitte formatiere deine Antwort als json-Datei wie folgt:
    {
        "aktenzeichen" : {extrahiertes Aktenzeichen},
        "name" : {extrahierter Name},
        "schadensnummer" : {extrahierte Schadensnummer},
        "kategorie" : {extrahierte Kategorie: Deckungszusage - Deckungsablehnung - Nachfrage - Kosten}
    }"""

    task_dz = """Bitte extrahiere folgende Informationen aus der Deckungszusage.
    Auf welche Instanz bezieht sich die Deckungszusage? Zur Auswahl steht das außergerichtliche Verfahren, das gerichtliche Verfahren der ersten Instanz und das
    Berufungsverfahren (2. Instanz).
    Was ist der Betrag der Selbstbeteiligung der Deckungszusage? Wenn keine Selbstbeteiligung erwähnt ist, antworte mit 0.

    Bitte formatiere deine Antwort als json-Datei wie folgt:
    {
        "außergerichtlich" : true/false,
        "gerichtlich" : true/false,
        "berufung" : true/false,
        "selbstbeteiligung" : {extrahierter Betrag}
    }
    """

    task_nachfrage = """1. Welche Nachfragegründe kannst du in der Nachfrage finden? Fragt die Rechtsschutzversicherung nach dem Sachstand? Fragt Sie nach der Klage?
    Fragt Sie nach dem Anspruchsschreiben? Erinnert Sie an ein vorheriges Schreiben? Fragt Sie nach sonstigen Fragen?

    2. Bitte füge alle gefundenen Antworten zu einer Liste zusammen.

    3. Bitte formatiere deine Antwort im json-Format wie folgt:
    {
        "Nachfragen" : [Liste der gefundenen Nachfragegründe: "Sachstand", "Klage", "Anspruchschreiben", "Erinnerung", "Sonstiges"]
    }
    """

    if step == 1:
        prompt = file_content + "-Ende der Nachricht-\n" + task1
    elif step == 2 and df["Typ"][idx] == "Deckungszusage":
        prompt = file_content + "-Ende der Deckungszusage-\n" + task_dz
    elif step == 2 and df["Typ"][idx] == "Nachfrage":
        prompt = file_content + "-Ende der Nachfrage-\n" + task_nachfrage

    return prompt

def send_request(prompt):
    response = llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
          {
              "role": "user",
              "content": prompt
          }
      ],
      max_tokens=2048,
      temperature=0.2,
    )
    return response

#convert response and save dict to df
def save_response(response, idx, step):
    column_name = f'response_{step}_{model}'
    if column_name not in df.columns:
        df[column_name] = None

    # llama and mistral
    if model == "llama-13b" or model == "mistral-7b" or model == "mixtral-8x7b":
        try:
            response_text = response['choices'][0]['message']['content'].strip()
            reponse_json  = extract_json(response_text)

            response_dict = json.loads(reponse_json)
            df[column_name][idx] = response_dict

            if len(response_dict) > 4:
                print("Unexpected number of arguments in dict!")
            else:
                for key, value in response_dict.items():
                    if f'{key}_{model}' not in df.columns:
                        df[f'{key}_{model}'] = ''
                    df.at[idx, f'{key}_{model}'] = value

        except:
            df[column_name][idx] = response_text
            print("Antwort konnte nicht zu json formatiert werden")



def extract_json(string):
    # Define a regular expression pattern to match JSON
    pattern = r'\{(.|\n)*\}'

    # Find the JSON part using regex
    json_match = re.search(pattern, string)

    if json_match:
        extracted_json = json_match.group()
        #print(extracted_json)
    else:
        extracted_json = "No JSON found"
        #print("No JSON part found in the string.")

    return extracted_json

In [6]:
# Imports
from llama_cpp import Llama
from huggingface_hub import hf_hub_download

In [7]:
modelpath = hf_hub_download(repo_id='TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF', filename="mixtral-8x7b-instruct-v0.1.Q5_K_M.gguf")

In [11]:
del llm

In [8]:
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path = modelpath, # Download the model file first
  n_ctx=1024,             # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=12,        # The number of layers to offload to GPU, if you have GPU acceleration available
  chat_format = "llama-2"
  #stop = "<</SYS>>"
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
prompt = """Nachricht:

Rechtsschutz-Schaden-Nr.: 2226-097.039.2-324
Ihr Aktenzeichen: CBC-TE3HYKD-442-BRUNNER
Unsere Kundin: Simone Brunner
Ereignis vom: 26. August 2022

Sehr geehrte Damen und Herren,
für die Interessenwahrnehmung erster Instanz besteht Versicherungsschutz.
Angesprochener Leistungsbereich ist der Rechtsschutz im Vertrags- und Sachenrecht.
Kosten bei einer einverständlichen Erledigung trägt der Versicherer, soweit die Kostenverteilung dem Verhältnis des angestrebten zum erzielten Erfolg entspricht. Das gilt nur dann nicht, wenn eine hiervon abweichende Kostenregelung gesetzlich vorgeschrieben ist. Im Falle eines gerichtlichen Vergleiches achten Sie bitte auf eine deutliche Einbeziehung etwaiger
Geschäftsgebühren. Etwaige (Mehr-) Kosten durch die Erledigung unstreitiger oder nicht versicherter Punkte sind nicht
rechtsschutzversichert

- Ende der Nachricht -

Bitte extrahiere folgende Informationen aus der Nachricht:
1. Um welche Kategorie von Dokument handelt es sich? Die Auswahlmöglichkeiten sind "Deckungszusage", "Deckungsablehnung", "Nachfrage" oder "Kostensache". Bei einer
Deckungszusage, erklärt die Versicherung, sie wird die zukünftigen Kosten des Falls übernehmen - bei einer Deckungszusage verneint sie das. Bei einer Nachfrage
möchte die Versicherung mehr Informationen von der Kanzlei. Eine Kostensache liegt nur vor, wenn die Versicherung erklärt, sie hat bereits Kosten bezahlt.
2. Wie lautet das Aktenzeichen der Kanzlei?
3. Wie lautet der Name des Mandanten?
4. Wie lautet die Schadensnummer oder Leistungsnummer des Versicherungsfalls?
Wenn du einen Wert nicht findest, schreibe "None".
Bitte formatiere deine Antwort als json-Datei wie folgt:
{
"aktenzeichen" : {extrahiertes Aktenzeichen},
"name" : {extrahierter Name},
"schadensnummer" : {extrahierte Schadensnummer},
"kategorie" : {extrahierte Kategorie: Deckungszusage - Deckungsablehnung - Nachfrage - Kosten (nur wenn bereits bezahlt)}
}"""

In [10]:
response = llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
        {
            "role": "user",
            "content": prompt
        }
    ]
)

In [11]:
print(response)

{'id': 'chatcmpl-4475bdd2-c31c-4d80-acd4-02a978b2f714', 'object': 'chat.completion', 'created': 1702980894, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Mixtral-8x7B-Instruct-v0.1-GGUF/snapshots/fa1d3835c5d45a3a74c0b68805fcdc133dba2b6a/mixtral-8x7b-instruct-v0.1.Q5_K_M.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': ' {\n"aktenzeichen": "CBC-TE3HYKD-442-BRUNNER",\n"name": "Simone Brunner",\n"schadensnummer": "2226-097.039.2-324",\n"kategorie": "Deckungszusage"\n}'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 704, 'completion_tokens': 78, 'total_tokens': 782}}


In [11]:
response['choices'][0]['message']['content'].strip()

'{\n"aktenzeichen": "CBC-TE3HYKD-442-BRUNNER",\n"name": "Simone Brunner",\n"schadensnummer": "2226-097.039.2-324",\n"kategorie": "Deckungszusage"\n}'

In [ ]:
model = "mixtral-8x7b"

for idx, row in df.iterrows():

    # first prompt/step
    prompt = generate_prompt(df, idx, step=1)
    response = send_request(prompt)

    print(response['choices'][0]['message']['content'])

    # write code to extract json only
    # and add it here
    save_response(response, idx, step=1)

    # second prompt/step only for "Deckungszusage" and "Nachfrage"
    if df["Typ"][idx] == "Deckungszusage" or df["Typ"][idx] == "Nachfrage":
        prompt2 = generate_prompt(df, idx, step=2)
        response2 = send_request(prompt2)
        # check if json only was returned
        save_response(response2, idx, step=2)

    print(f"Successfully processed entry {idx +1} of {df.shape[0]}")

    if idx >= 2:
        break

df.head(2)

Llama.generate: prefix-match hit


 {
"aktenzeichen": "None",
"name": "Denver Pearson",
"schadensnummer": "S-21-03384164",
"kategorie": "Nachfrage"
}


Llama.generate: prefix-match hit
